In [8]:
#New version of numpy is there, using the upgraded one
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random
import matplotlib.pyplot as plt
%matplotlib inline

from itertools import cycle

from math import ceil

from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler

warnings.filterwarnings('ignore')

#the sales training data has more than 1900 columns
pd.set_option('max_columns', 50)
#These are used for better plotting, check them later or ask about them
plt.style.use('bmh')
color_pal = plt.rcParams['axes.prop_cycle'].by_key()['color']
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

In [11]:
input_dir = '/home/aviral/Desktop/retail/m5-forecasting-accuracy/'

#CHECK AGAIN HOW THIS IS WORKING, IS IT ZIP MECHANISM OR SOMETHING ELSE
def reduce_mem_usage(df, verbose=False):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics: 
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [36]:
def read_data(input_dir):
    sell_prices_df = pd.read_csv(input_dir+'sell_prices.csv')
    sell_prices_df = reduce_mem_usage(df=sell_prices_df, verbose=True)
    print('The shape of sell prices is ', sell_prices_df.shape)
    
    sales_train_validation_df = pd.read_csv(input_dir+'sales_train_validation.csv')
    sales_train_validation_df = reduce_mem_usage(df=sales_train_validation_df, verbose=True)
    print('The shape of sales data is ', sales_train_validation_df.shape)
    
    calendar_df = pd.read_csv(input_dir+'calendar.csv')
    calendar_df = reduce_mem_usage(df=calendar_df, verbose=True)
    print('The shape of the calendar is ', calendar_df.shape)
    
    return sell_prices_df, sales_train_validation_df, calendar_df
    

In [37]:
t1 = time.time()
sell_df, sales_df, calendar = read_data(input_dir)
t2 = time.time()
print('Time taken for loading data is ', (t2-t1))

Mem. usage decreased to 130.48 Mb (37.5% reduction)
The shape of sell prices is  (6841121, 4)
Mem. usage decreased to 95.00 Mb (78.7% reduction)
The shape of sales data is  (30490, 1919)
Mem. usage decreased to  0.12 Mb (41.9% reduction)
The shape of the calendar is  (1969, 14)
Time taken for loading data is  91.17874264717102


In [38]:
#Ask rahul about the second column, what does the middle digit represent
calendar.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [39]:
sales_df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,...,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,4,2,3,0,1,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,3,1,2,1,3,1,0,2,5,4,2,0,3,0,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,4,0,1,4,0,1,0,1,0,1,1,2,0,1,1,2,1,1,0,1,1,2,2,2,4


In [40]:
sell_df.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.578125
1,CA_1,HOBBIES_1_001,11326,9.578125
2,CA_1,HOBBIES_1_001,11327,8.257812
3,CA_1,HOBBIES_1_001,11328,8.257812
4,CA_1,HOBBIES_1_001,11329,8.257812


In [24]:
import datetime
#Create a dates list, this will be used in plotting later
date_index = calendar['date']
#Even though the calendar has 1969 dates, the sales data is available for only 1913 days
#The rest of it might be for validation and testing
dates = date_index[0:1819]
dates_list = [datetime.datetime.strptime(date, '%Y-%m-%d').date() for date in dates]

In [32]:
#This is the aggregated per category and per store
data = pd.DataFrame(sales_df.groupby(by = ['cat_id', 'dept_id', 'item_id', 'store_id']).sum())
data.head(n=15)

d_1  d_2  d_3  d_4  d_5  d_6  d_7  d_8  \
cat_id dept_id item_id     store_id                                           
FOODS  FOODS_1 FOODS_1_001 CA_1        3    0    0    1    4    2    0    2   
                           CA_2        2    0    0    0    1    0    8    0   
                           CA_3        1    2    1    1    1    2    0    1   
                           CA_4        0    1    1    1    1    1    0    0   
                           TX_1        0    1    1    0    0    0    0    0   
                           TX_2        0    0    1    2    0    4    0    0   
                           TX_3        0    0    0    0    0    0    0    0   
                           WI_1        0    2    0    1    0    5    0    0   
                           WI_2        0    0    0    0    0    1    2    1   
                           WI_3        0    0    0    0    0    3    0    0   
               FOODS_1_002 CA_1        0    1    0    1    1    1    0    0   
                           CA_2        2    1    1    0    2    2    0    1   
                           CA_3        1    0    1    2    0    0    0    1   
                           CA_4        0    1    2    1    0    0    0    0   
                           TX_1        0    0    0    0    0    0    0    0   

                                     d_9  d_10  d_11  d_12  d_13  d_14  d_15  \
cat_id dept_id item_id     store_id                                            
FOODS  FOODS_1 FOODS_1_001 CA_1        0     0     0     0     3     1     3   
                           CA_2        4     1     2     1     1     3     4   
                           CA_3        1     1     0     0     3     3     2   
                           CA_4        0     0     0     0     1     2     1   
                           TX_1        2     1     0     0     0     1     0   
                           TX_2        2     2     0     1     2     0     0   
                           TX_3        0     0     4     1     0     0     0   
                           WI_1        1     1     0     0     1     0     2   
                           WI_2        1     1     0     0     0     1     2   
                           WI_3        0     3     0     2     1     4     2   
               FOODS_1_002 CA_1        0     0     1     1     0     2     0   
                           CA_2        0     0     1     0     0     0     5   
                           CA_3        1     0     1     0     0     0     1   
                           CA_4        0     1     0     1     0     0     0   
                           TX_1        0     0     0     1     0     2     1   

                                     d_16  d_17  d_18  d_19  d_20  d_21  d_22  \
cat_id dept_id item_id     store_id                                             
FOODS  FOODS_1 FOODS_1_001 CA_1         0     2     1     2     0     2     1   
                           CA_2         1     2     2     2     0     1     1   
                           CA_3         7     1     2     4     0     1     1   
                           CA_4         0     1     0     0     1     0     2   
                           TX_1         0     0     0     0     0     0     0   
                           TX_2         4     4     1     0     1     2     0   
                           TX_3         0     0     2     1     1     0     1   
                           WI_1         0     0     1     2     0     7     0   
                           WI_2         0     1     1     1     1     4     1   
                           WI_3         6     0     0     0     3     0     2   
               FOODS_1_002 CA_1         1     0     0     0     0     0     0   
                           CA_2         3     0     0     0     0     0     3   
                           CA_3         0     0     0     0     0     1     0   
                           CA_4         0     0     0     1     0     0     0   
                           TX_1     

In [31]:
#The complete sales for the 6 year time period of each item in each of the stores

sales = pd.DataFrame(data.sum(axis = 1))
sales.rename(columns = {0:'sales'},inplace=True)
sales.head(15)

sales
cat_id dept_id item_id     store_id       
FOODS  FOODS_1 FOODS_1_001 CA_1       1503
                           CA_2       2214
                           CA_3       2302
                           CA_4        688
                           TX_1       1119
                           TX_2       1112
                           TX_3        785
                           WI_1       1086
                           WI_2        875
                           WI_3        563
               FOODS_1_002 CA_1        914
                           CA_2       1154
                           CA_3        603
                           CA_4        640
                           TX_1        323

In [35]:
#Same procedure as above, but now the tables are store-wise instead of item-wise
stores_df = pd.DataFrame(sales_df.groupby(by = ['store_id', 'cat_id']).sum())

stores = pd.DataFrame(stores_df.sum(axis=1))
stores.rename(columns = {0:'Total sales'}, inplace=True)
stores

Total sales
store_id cat_id                
CA_1     FOODS        5380828.0
         HOBBIES       876678.0
         HOUSEHOLD    1440710.0
CA_2     FOODS        3480259.0
         HOBBIES       637803.0
         HOUSEHOLD    1567413.0
CA_3     FOODS        7515790.0
         HOBBIES       960947.0
         HOUSEHOLD    2711443.0
CA_4     FOODS        2819425.0
         HOBBIES       564455.0
         HOUSEHOLD     719796.0
TX_1     FOODS        3778209.0
         HOBBIES       429084.0
         HOUSEHOLD    1387999.0
TX_2     FOODS        5015124.0
         HOBBIES       635997.0
         HOUSEHOLD    1563263.0
TX_3     FOODS        4163088.0
         HOBBIES       527258.0
         HOUSEHOLD    1398984.0
WI_1     FOODS        3438352.0
         HOBBIES       655696.0
         HOUSEHOLD    1055014.0
WI_2     FOODS        4768184.0
         HOBBIES       370214.0
         HOUSEHOLD    1405614.0
WI_3     FOODS        4730680.0
         HOBBIES       466668.0
         HOUSEHOLD    1230434.0

In [ ]:
import seaborn as sns
